In [54]:
import regex as re
import pandas as pd
import os
import random
import numpy as np
from imblearn.over_sampling import SMOTE
import warnings
warnings.filterwarnings("ignore", category=UserWarning, module='your module name')

In [55]:
data = pd.read_csv('smalldata_washed.csv')
rumor = data['rumor'].to_list()
reverse = data['reverse'].to_list()
rumor_class = len(rumor)*[1]
reverse_class = len(reverse)*[0]
data = rumor + reverse
data_class = rumor_class + reverse_class
#后续添加数据要放在raw目录下，使用.xlsx格式将后续标注的数据加入data
raw_data_list = os.listdir('raw')
data_list = []
good_name = re.compile(r'^(?!(\~\$)).*(.xlsx)')
for i in raw_data_list:
    if good_name.match(i):
    #    data_list.append(i)
        temp = pd.read_excel('raw/'+i)
        temp.fillna('',inplace=True)
        temp_rumor = [x.strip() for x in  temp['punc_rumor'].to_list()  if x.strip()!='']
        temp_reverse = [x.strip() for x in  temp['punc_truth'].to_list()  if x.strip()!='']
        temp_rumor_class = len(temp_rumor)*[1]
        temp_reverse_class = len(temp_reverse)*[0]
        temp_data = temp_rumor + temp_reverse
        temp_data_class = temp_rumor_class + temp_reverse_class
        for i in range( len(temp_data)):
            temp_data[i]=re.sub(r'[\,\'\ ]|(\n)','',temp_data[i])
    data = data + temp_data
    data_class = data_class + temp_data_class
with open('data.txt','w') as f:
    for i in range(len(data)):
        f.write(data[i]+'\t'+str(data_class[i])+'\n')
        data_list =list( zip(data, data_class))
random.shuffle(data_list)
data, data_class = zip(*data_list)
index = ['rumor{}{}'.format(i, '!' * j) for (i, j) in zip(range(len(data)), data_class)]
df = pd .DataFrame(data=zip(data_class,data), columns=["rumor","text"], index=index)

# 去除停用词
with open ('hit_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = f.read().splitlines()

# jieba分词
import jieba
for i in range(len(df)):
    words = jieba.cut(df["text"].iloc[i],cut_all=False)
    words = [word for word in words if word not in stopwords]
    df["text"].iloc[i] = ' '.join(words) 

/var/folders/f0/6j6j3w814n53skbt2s4d8p5m0000gn/T/ipykernel_36238/2493417845.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["text"].iloc[i] = ' '.join(words)


#### 隐形狄利克雷分布原理
- 设想一台自动生成文档的机器，只有两个选项来控制生成文档的两个属性
    - 生成文档的词的数量（泊松分布）
    - 文档中混合的主题的数量（狄利克雷分布）
- 除此之外，最关键的问题在于确定一个主题-词项矩阵，该矩阵表示了每个词对主题的贡献权重。一但能够确定这个矩阵，机器就可以在选择好的主题上反复迭代选择词，直到生成一篇足够长度的文档。
- 回到对现有文档估计主题的问题上来，LDiA可以用于关于词和主题的参数，Blei和Ng通过分析语料库中文档的统计数据确定两个参数
    - 第一个参数可以通过计算语料库的平均词（n-gram）数量、
    - 第二个参数更棘手，必须先猜测有几个主题数再为主题分配语词，最后优化目标函数
-  “LSA试图将原本分散的东西分散开，LDiA试图将原本接近的东西接近在一起”

In [56]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from nltk.tokenize import casual_tokenize
np.random.seed(42)

counter = CountVectorizer(tokenizer=casual_tokenize)
bow_docs = pd.DataFrame(counter.fit_transform(raw_documents=df.text)
                        .toarray(), index=index)
column_nums, terms = zip(*sorted(zip(counter.vocabulary_.values(),
                                     counter.vocabulary_.keys())))
bow_docs.columns = terms

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [57]:
try:
    print('rumor0:'+df.loc['rumor0'].text)
    print(bow_docs.loc['rumor0'][bow_docs.loc['rumor0'] > 0].head())
except KeyError:
    print('rumor0!:'+df.loc['rumor0!'].text)
    print(bow_docs.loc['rumor0!'][bow_docs.loc['rumor0!'] > 0].head())

rumor0:已经 植入 心脏 支架 患者 一定 坚持 正规 西药 抗 血小板 治疗 不可 中药 代替 支架 内 发生 再 狭窄 堵塞 概率 将会 极大 增加 诱发 急性 心肌梗死
一定    1
不可    1
中药    1
代替    1
内     1
Name: rumor0, dtype: int64


In [58]:
from sklearn.decomposition import LatentDirichletAllocation as LDiA
ldia = LDiA(n_components=16, learning_method='batch')
ldia = ldia.fit(bow_docs)  # <1>
ldia.components_.shape

(16, 9142)

In [59]:
columns = ['topic{}'.format(i) for i in range(ldia.components_.shape[0])]
pd.set_option('display.width', 2)
componets = pd.DataFrame(ldia.components_.T,index = terms ,columns=columns)
componets.round(2).head(3)

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
%,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,1.06,0.06
0,0.06,0.06,0.06,1.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06
0t,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,0.06,2.06,0.06,0.06,0.06,0.06,0.06,0.06


In [60]:
componets.topic0.sort_values(axis=0, ascending=False).head(10)

会     200.351333
出现    106.214661
都      86.943403
可能     77.964169
不      68.163753
这种     56.155247
心脏     55.547909
症状     54.703698
头痛     53.381958
没有     50.712851
Name: topic0, dtype: float64

In [61]:
ldia16_topic_vectors = ldia.transform(bow_docs)
ldia16_topic_vectors = pd.DataFrame(ldia16_topic_vectors,
                                    index=index, columns=columns)
ldia16_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,topic10,topic11,topic12,topic13,topic14,topic15
rumor0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.21,0.46,0.0,0.0,0.00,0.31,0.0,0.0
rumor1,0.00,0.0,0.0,0.0,0.0,0.83,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.13,0.0,0.0
rumor2,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.28,0.00,0.0,0.0,0.00,0.00,0.0,0.7
rumor3,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.00,0.00,0.0,0.0,0.99,0.00,0.0,0.0
rumor4,0.75,0.0,0.0,0.0,0.0,0.23,0.0,0.0,0.00,0.00,0.0,0.0,0.00,0.00,0.0,0.0


#### 使用LDiA主题向量训练LDA模型

In [62]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(ldia16_topic_vectors,df.rumor,test_size=0.2,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA()
lda = lda.fit(X_train_res, y_train_res)
df['ldia16_rumor'] = lda.predict(ldia16_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 1576
过采样前训练集中类别1的数量： 416
过采样后训练集中类别0的数量： 1576
过采样后训练集中类别1的数量： 1576


0.538

In [65]:
print(y_test.index)

Index(['rumor1057',
       'rumor478',
       'rumor976',
       'rumor422!',
       'rumor1399',
       'rumor1447',
       'rumor2329!',
       'rumor1937!',
       'rumor2064!',
       'rumor2476',
       ...
       'rumor1847',
       'rumor1146!',
       'rumor461',
       'rumor109',
       'rumor1943!',
       'rumor354',
       'rumor266!',
       'rumor889!',
       'rumor1553',
       'rumor564'],
      dtype='object', length=498)


In [66]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].ldia16_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["非rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
print(confusion)

              precision    recall  f1-score   support

      非rumor     0.8197    0.5040    0.6242       379
       rumor     0.2906    0.6471    0.4010       119

    accuracy                         0.5382       498
   macro avg     0.5552    0.5755    0.5126       498
weighted avg     0.6933    0.5382    0.5709       498

[[191 188]
 [ 42  77]]


#### 对比TF-IDF向量的分类

In [46]:
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import casual_tokenize
tfidf = TfidfVectorizer(tokenizer=casual_tokenize,)
tfidf_docs =tfidf.fit_transform(raw_documents=df.text).toarray()
tfidf_docs = tfidf_docs - tfidf_docs.mean(axis=0)

/Users/stevenyu/opt/anaconda3/envs/d2l/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [47]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(tfidf_docs,df.rumor,test_size=0.2,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA()
lda = lda.fit(X_train_res, y_train_res)
df['tfidf_rumor'] = lda.predict(tfidf_docs)

过采样前训练集中类别0的数量： 1558
过采样前训练集中类别1的数量： 434
过采样后训练集中类别0的数量： 1558
过采样后训练集中类别1的数量： 1558


In [48]:
round(float(lda.score(X_test, y_test)), 3)

0.789

In [49]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].tfidf_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["非rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
print(confusion)

              precision    recall  f1-score   support

         非谣言     0.9827    0.9591    0.9707      1955
          谣言     0.8625    0.9383    0.8988       535

    accuracy                         0.9546      2490
   macro avg     0.9226    0.9487    0.9348      2490
weighted avg     0.9569    0.9546    0.9553      2490

[[1875   80]
 [  33  502]]


#### LDiA使用32个主题

In [50]:
ldia32 = LDiA(n_components=100, learning_method='batch')
ldia32 = ldia32.fit(bow_docs)
ldia32.components_.shape

(100, 9142)

In [51]:
ldia32_topic_vectors = ldia32.transform(bow_docs)
columns32 = ['topic{}'.format(i) for i in range(ldia32.components_.shape[0])]
ldia32_topic_vectors = pd.DataFrame(ldia32_topic_vectors,index=index, columns=columns32)
ldia32_topic_vectors.round(2).head()

,topic0,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,topic90,topic91,topic92,topic93,topic94,topic95,topic96,topic97,topic98,topic99
rumor0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
rumor1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
rumor2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0
rumor3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.09,0.0
rumor4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,0.0


In [52]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(ldia32_topic_vectors,df.rumor,test_size=0.2,random_state=42)
smote = SMOTE(random_state=42)
# 进行过采样
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)
print("过采样前训练集中类别0的数量：", sum(y_train==0))
print("过采样前训练集中类别1的数量：", sum(y_train==1))

print("过采样后训练集中类别0的数量：", sum(y_train_res==0))
print("过采样后训练集中类别1的数量：", sum(y_train_res==1))
lda = LDA()
lda = lda.fit(X_train_res, y_train_res)
df['ldia32_rumor'] = lda.predict(ldia32_topic_vectors)
round(float(lda.score(X_test, y_test)), 3)

过采样前训练集中类别0的数量： 1558
过采样前训练集中类别1的数量： 434
过采样后训练集中类别0的数量： 1558
过采样后训练集中类别1的数量： 1558


0.512

In [53]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import metrics
import time
from tensorboardX import SummaryWriter

y_true = df.loc[y_test.index].rumor
y_pred = df.loc[y_test.index].ldia32_rumor
loss_total = 0
acc = metrics.accuracy_score(y_true, y_pred)
report = metrics.classification_report(y_true, y_pred, target_names=["non-rumor","rumor"], digits=4)
confusion = metrics.confusion_matrix(y_true, y_pred)
print(report)
print(confusion)

              precision    recall  f1-score   support

         非谣言     0.8527    0.6010    0.7051      1955
          谣言     0.2986    0.6206    0.4032       535

    accuracy                         0.6052      2490
   macro avg     0.5756    0.6108    0.5541      2490
weighted avg     0.7336    0.6052    0.6402      2490

[[1175  780]
 [ 203  332]]
